In [ ]:
#Limit dates to plot
N_days=60 # number of days to include
bad_cutoff=20000 # minimum number of tests to consider good


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import requests
import zipfile
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import date, timedelta

In [ ]:
#Define date of today used format
d=date.today()
date_str=d.strftime("%d%m%Y")

#Define a manual date (e.g. if running on a date where no new data is uploaded)
#date_str="13112020"

In [ ]:
# Find link to latest file from the web page
# assumes stabel initial part of filename and webpage location
#req = Request("https://www.ssi.dk/sygdomme-beredskab-og-forskning/sygdomsovervaagning/c/covid19-overvaagning")
req = Request("https://covid19.ssi.dk/overvagningsdata/download-fil-med-overvaagningdata")
html_page = urlopen(req)
soup = BeautifulSoup(html_page, "html")
                     
for link in soup.findAll('a'):
    ref=link.get('href')
    if isinstance(ref, str) and "data-epidemiologiske-rapport" in ref.lower() and date_str in ref.lower(): # NB compare in lower case
        print("url for zip file: " + link.get('href'))
        url=link.get('href')
        #date_str = url[50:58]
    if isinstance(ref, str) and "data-epidemiologisk-rapport-" in ref.lower() and date_str in ref.lower(): # NB compare in lower case
        print("url for zip file: " + link.get('href'))
        url=link.get('href')
        #date_str = url[49:57]

        



In [ ]:
#Defin todays URL manualy if above code breaks
#url = 'https://files.ssi.dk/Data-epidemiologisk-rapport-09112020-1puf'
#url='https://files.ssi.dk/covid19/overvagning/data/data-epidemiologiske-rapport-12112020-ql82'



In [ ]:
# Direct download data and unpacking
r = requests.get(url, allow_redirects=True)
open('data.zip', 'wb').write(r.content)

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall("data/")

#define the file to read
datafolder=Path("data/")
datafile=datafolder / "Test_pos_over_time.csv"

In [ ]:
# Read datafile
# Skips last two lines (which does not convert to date) and converts index to date
# Notice handeling of danish format of the numbers (both decimal and thousands)
df=pd.read_csv(datafile,  sep=';', parse_dates=['Date'], index_col=['Date'],error_bad_lines=False, engine='python', skipfooter=2, decimal=',', thousands='.')


In [ ]:
# calculate some more numbers

# Positive emperical scaled by number of tests to power of 0.7  
# This scaling is based on results in 
# SSI "Ekspertrapport af d. 23. oktober 2020 Incidens og fremskrivning af COVID-19 tilfælde"
# https://www.ssi.dk/-/media/ssi-files/ekspertrapport-af-den-23-oktober-2020-incidens-og-fremskrivning-af-covid19-tilflde.pdf?la=da
def calcScaledNumber (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / (row.NotPrevPos**0.7) * 50000**0.7 / 50000 *100#Normalized positiv procent to 50000 tests
    else:
        return 0
    
df['ScaledNumber']=df.apply(lambda row: calcScaledNumber(row), axis=1)    

# Recalculate Positiv procent to get more decimals for plotting
def calcPosPct (row):
    if row.NotPrevPos > 0 :
        return row.NewPositive / row.NotPrevPos * 100
    else:
        return 0
df['PosPct']=df.apply(lambda row: calcPosPct(row), axis=1)


In [ ]:
# for easy plot make a sub data frame with selected number of days 
df_sel=df[date.today()-timedelta(days=N_days):]

# and make index for "bad" datapoints
bad_idx=df_sel['NotPrevPos']<bad_cutoff

In [ ]:
# define a common title including date from file
title_str='SSI COVID-19 data, tilfælde opgjort på prøvetagningsdato \n' 
title_str += date_str

In [ ]:
ax=df_sel.plot(y='NewPositive',title=title_str,style='.');
df_sel[bad_idx].plot(ax=ax,y='NewPositive',style='.',color='red',label='NewPositive (Tested<'+ str(bad_cutoff) + ')');


In [ ]:
ax=df_sel.plot(y='NotPrevPos',label='Tested (NotPrevPos)',title=title_str,style='.');
df_sel[bad_idx].plot(ax=ax,y='NotPrevPos',style='.',color='red',label='Tested<'+ str(bad_cutoff) + '');

In [ ]:
ax=df_sel.plot(y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.');
df_sel[bad_idx].plot(ax=ax,y='PosPct',style='.',color='red',label='NewPositive / NotPrevPosTested * 100 (Tested<'+ str(bad_cutoff) + ')');
ax.set_ylabel("%");


In [ ]:
ax=df_sel.plot(y='ScaledNumber',title=title_str,label='NewPositive/NotPrevPosTested^0.7',style='.');
df_sel[bad_idx].plot(ax=ax,y='ScaledNumber',style='.',color='red', label='NewPositive/NotPrevPosTested^0.7 (Tested<'+ str(bad_cutoff) + ')');


In [ ]:
axs=[None]*3 #define axs list as empty 3 entries
fig = plt.figure(figsize=(7, 10))
axs[0] = plt.subplot(311)
axs[1] = plt.subplot(312,sharex=axs[0])
axs[2] = plt.subplot(313,sharex=axs[0])


df_sel.plot(ax=axs[0],y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.');
df_sel[bad_idx].plot(ax=axs[0],y='PosPct',style='.',color='red',label='NewPositive / NotPrevPosTested * 100 (Tested<'+ str(bad_cutoff) + ')');
axs[0].set_ylabel("%");
axs[0].set_ylim(0,2)

df_sel.plot(ax=axs[1],y='NewPositive',style='.');
df_sel[bad_idx].plot(ax=axs[1],y='NewPositive',style='.',color='red',label='NewPositive (Tested<'+ str(bad_cutoff) + ')');

df_sel.plot(ax=axs[2],y='NotPrevPos',label='Tested (NotPrevPos)',style='.');
df_sel[bad_idx].plot(ax=axs[2],y='NotPrevPos',style='.',color='red',label='Tested<'+ str(bad_cutoff) + '');

In [ ]:
axs=[None]*4 #define axs list as empty 4 entries
fig = plt.figure(figsize=(7, 15))
axs[0] = plt.subplot(411)
axs[1] = plt.subplot(412,sharex=axs[0])
axs[2] = plt.subplot(413,sharex=axs[0])
axs[3] = plt.subplot(414,sharex=axs[0])


df_sel.plot(ax=axs[0],y='PosPct',title=title_str,label='NewPositive / NotPrevPosTested * 100',style='.');
df_sel[bad_idx].plot(ax=axs[0],y='PosPct',style='.',color='red',label='NewPositive / NotPrevPosTested * 100 (Tested<'+ str(bad_cutoff) + ')');
axs[0].set_ylabel("%");
axs[0].set_ylim(0,2)

df_sel.plot(ax=axs[1], y='ScaledNumber',label='NewPositive/NotPrevPosTested^0.7 * 50.000^0.7 / 50.000 *100',style='.');
df_sel[bad_idx].plot(ax=axs[1],y='ScaledNumber',style='.',color='red', label=' (Tested<'+ str(bad_cutoff) + ')');
axs[1].set_ylabel("Positiv Procent [Estimated for 50.000 tests]");
axs[1].set_ylim(0,2)


df_sel.plot(ax=axs[2],y='NewPositive',style='.');
df_sel[bad_idx].plot(ax=axs[2],y='NewPositive',style='.',color='red',label='NewPositive (Tested<'+ str(bad_cutoff) + ')');

df_sel.plot(ax=axs[3],y='NotPrevPos',label='Tested (NotPrevPos)',style='.');
df_sel[bad_idx].plot(ax=axs[3],y='NotPrevPos',style='.',color='red',label='Tested<'+ str(bad_cutoff) + '');

#save a pdf for printing
plt.savefig('All4.pdf')  